# Geospatial Analysis of Los Angeles 

### In this notebook, we leverage Foursquare location data to aid us in better understanding the cities in Los Angeles

#### Introduction
We try to aid people who are thinking of migrating to Los Angeles to better understand the cities, to decide which area may be best suited for them to move into. We assume that an important feature that people consider when making a decision is the type of venues available within the city.

#### Data
The data that we'll be using will mainly come from exploring Foursquare's data of Los Angeles, and along the way we might include other data sources into this workbook. For Foursquare's data, we will look into the restaurants/landmarks within each city, to find distinct characteristics within the city that we may be able to cluster together.